<h1 style="color:blue">Praktikum 8. Masinõpe</h1>
<h3 style="color:blue">Praktiline masinõpe - klassifitseerimine 2</h3>
<h4>25. oktoober 2016 </h4>
<h4>Ülesannete esitamise tähtaeg 8. november.</h4>

Klassifitseerimine on tekstiga seotud lahendustes olulisel kohal. Me tahame sageli automaatselt lahterdada suurt hulka dokumente mingi kriteeriumi kohaselt - olgu selleks spämmi ja inimkaubitsejate tuvastamine või veebipoe toodete liigitamine.

Eelmisel korral mängisime "lihtsamate" tõenäosuslike mudelitega. Käesolevas praktikumis tutvume lisaks uutele meetoditele ka andmete eripärade ja teiste oluliste klassifitseerimisprobleemidega keeles. Esmalt tutvume meelestatuse analüüsiga.

## Ülesanne 1. Meelestatuse analüüs (5,5+p)

Meelestatuse analüüs *(sentiment analysis)* püüab kirjeldada, millise "meelestatusega" on tekst kirjutatud. Kõige lihtsamal juhul on tekst kirjeldatu suhtes kas positiivne või negatiivne ehk meie probleem
taandub binaarseks klassifitseerimiseks. Kui loome investorite meelitamiseks/klientide soovide kompamiseks [MVP](https://en.wikipedia.org/wiki/Minimum_viable_product) või artikliks *baseline*'i, peaksime minema kõige lihtsamat teed ja kasutama juba omandatud teadmisi. Selleks ehitame naiivse klassifitseerija, mis üritab tuvastada positiivsust ja negatiivsust kirjeldavaid sõnu, n-gramme või muid tunnuseid. Tuletame meelde, et klassifitseerimine on juhendatud õppimine, mis tähendab, et ennustava mudeli treenimisel on meil tarvis hulka andmeid koos meelestatuse hinnangutega, mida õpetame mudelit ennustama.

**Andmed.** Failis *hinnavaatlus.json* on toodud 44 firma teenuse osutamise kvaliteedi kohta käivad kommentaarid 2015. aasta oktoobri seisuga koos kommenteerija hinnanguga skaalal 1-5. Failis on ühel real JSON-formaadis objektide listina talletatud kommentaarid, kusjuures igal kommentaariobjektil on olemas
- kommentaari sisu - “comment”,
- kommenteerija antud hinnang - “rating”,
- kommenteeritud firma - “company”.

#### Alustame andmete eeltöötlemisega. (0.5p)
Selleks loeme esmalt sisse faili *hinnavaatlus.json* ning loome sellest vastava Püütoni sõnaraamatu. 

In [1]:
import json

# Store hinnavaatlus.json data in a variable

Järgmise sammuna eraldame kommentaari sisust tunnused. Seda on ilus teha funktsioonina, mis tagastab kommentaari tekstile vastava *list*'i või *tuple*'i tunnustest. Milliseid tunnuseid ja kui palju Te kasutate, on Teie valida - väheste ja ebaolulistega on aga ühtsoodu igav ning tulemused on vapustavalt halvad. Loodav tunnusvektor võib koosneda näiteks üksikute lemmade ja 2-grammide sagedustest koos kommentaari ja keskmise sõna pikkusega.

<h5 style="color:red">Kui eelemises praktikumis sai uhkust valmistav tunnuste eraldaja kirjutatud, saab sealse koodiploki talleteda %file faili_nimi.py maagia abil Püütoni moodulina, mille saame nüüd importida. Ideaalis võib kasutada Püütoni <i>package</i>'eid.</h5>

In [ ]:
def extract_features(comment):
    return ()

Loome kommentaaride tunnusvektorid (maatriksina) ning neile vastava reitingute vektori (esimesel real olevale tunnusvektorile vastab reitingute vektori esimene element jne). Kuivõrd tegu on reaalse väikese veebist kraabitud andmestikuga, ei ole see ideaalne. See tähendab, et kommentaarid võivad olla ebapädevad, reitingud puududa ning positiivsete ja negatiivsete vaatluste arv ei pruugi olla tasakaalus. Seega peab ideaalsel juhul enne treenima hakkamist andmetega veidi vaeva nägema.

*pandas*'e teek *DataFrame*'iga võib teha protsessi mugavamaks. Näiteks saab kergelt tuvastada puuduvad väärtused ning nendega midagi ette võtta. Lahendada saab aga ka ilma teegita.

Andmete ja jaotuste parandamisel annab väga loovaks minna. Kui "tavalistes" andmestikes mingid sõltumatud tunnused puuduvad, võib neid üritada näiteks tagasi ennustada, kasutades rohkete/piiramatute väärtuste korral mõnda lihtsat regressiooni meetodit, mõningate väärtuste korral aga mõnda klassifitseerimismeetodit - näiteks *Naive Bayes*'i.

Andmeid eemaldatakse andmestikust vaid juhul, kui neid on väga palju ning seda võib lubada (lugeda: kogu tunnusruum on küllaga erinevates piirkondades kaetud) või kui neist pole reaalselt kasu. Viimasel juhul näiteks tekstide korral, kui me pole täiesti kindlad oma sõltumatute tunnuste pädevuses ning ennustatav sõltuv tunnus on puudu. Sellisel puhul ei saa me kasutada vaatlust ei treenimisel, testimisel ega valideerimisel. Küll on aga võimalik vaatlust kasutada jätkuiteratsioonidel, kui oleme mingi esmase mudeli loonud ning suudame-julgeme selle abil täita varasemalt puuduvad lüngad ning uue informatsiooni valguses loodame paremaid tulemusi.

Kui meie andmestikus pole aga ennustatavad klassid ühtlaselt jaotunud, on kallutatud andmetel treenitud masinõppemeetodid kallutatud ning õpivad suuresti andmestiku iseärasust. Kui meil on binaarne klassifitseerimine, andmestikus klassid jaotunud 9:1 ning treening- ja testandmestikud juhuslikult valitud, võime oodata 90% täpsust puhtalt [andmestiku jaotusest](https://en.wikipedia.org/wiki/Prior_probability). See ei vasta aga harilikult reaalsusele, mistõttu valim peab olema juhuslik ehk andmestik peab olema mitmekesine.

Klassifitseerimisel on jaotuse probleemide lahendamiseks palju erinevaid võtteid, millest mõningaid kirjeldatakse [siin](http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/). Parim on alati koguda rohkem andmeid, et klassid ühtlustuksid. Kui see pole võimalik, saab vähemesinevat klassi *oversample*'ida (valida sealt korduvalt vaatlusi) või sagedamini esinevat klassi *undersample*'ida (valida vaatluste alamhulk). Viimasega peab olema ettevaatlik, et andmeid liiga väheks ei jääks tunnusruumi suhtes. Lisaks on võimalik treenida mudelite ansambel, nii et vähem esindatud klassi andmed on fikseeritud ning erinevatel mudelitel on erinevad rohkem esindatud klasside andmete alamhulgad. Kui meil on piisav *domain knowledge* olemas, saame vahel ka juua tehislikke vaatlusi.

*High-level pandas*'e asemel saame kasutada andmestike loomisel ka *numpy*'t (millel *pandas* nagunii baseerub). *Numpy* võimaldab luua fikseeritud suurusega ning rangelt tüübitud [*n*-dimensionaalseid vektoreid](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html), millel annab rakendada kõikvõimalikke lineaaralgebrast tuttavaid operatsioone C-s kirjutatud implementatsioonidel.

Järgneb näide, kuidas luua tunnusmaatrikseid ja märgendite vektoreid.

In [2]:
import numpy as np

raw_features = [
    (1, 2.7, 0, 4),
    (4, 3.2, 0.17, 1)
]

raw_labels = [0,1]

features = np.array(raw_features)
labels = np.array(raw_labels)

In [ ]:
# My pretty feature matrices and labels

### Boonusülesanne 1. Üritage lahendada jaotuse probleem andmestiku suurendamise abil. Veebikraapimine peaks algama [siit](http://www.hinnavaatlus.ee/firms/). (1p)

#### Jätkame andmete töötlemisega treenimise tarbeks. (0.5p)

Teisendame esmalt reitingu binaarsele (heale) kujule. Selleks seame reitingud 1-4 -> 0 (halb) ja 5 -> 1 (hea).

In [ ]:
# My new label vector

Jagame andmestiku treening-, valideerimis- ja testandmestikuks (80%-5%-15%).

In [3]:
# My 6 variables holding training, validation, and test features and labels

### Boonusülesanne 2. Lahendage 3 andmestiku leidmine kahe *cv.train_test_split* kutsega. (0.1p)

**1B. (4p)** Kirjutage kood, mille ülesandeks on:

**Lugeda ja töödelda andmed (0,5p)**
* lugeda sisse kettale salvestatud Numpy struktuurid,
* teisendada reiting binaarsele kujule, kus reiting 1-4 -> 0 (halb) ja reiting 5 -> 1 (hea)
* jagada andmestik treening-, valideerimis- ja testandmestikuks (nt 70%-10%-20%)

### Erinevate meetodite treenimine ning tulemuse hindamine
Järgnevalt treenime koos hüperparameetrite tuunimisega erinevaid praktikas rohket kasutust leidvate meetodite abil mudeleid ning võrdleme nende tulemusi. Tuletame meelde, et hüperparameetrid on mudeli arvutamisel arvesse võetavad väärtused, mida mudeli treenimise käigus ei õpita.

#### Juhuslik mets (*random forest*) (1,5p)
Juhuslik mets on palju rakendust leidev meetod, mis saab mingi maani hakkama suurte dimensioonidega (eeldusel, et andmeid on piisavalt) ning on oma ansambli meetodi tõttu tulemusi andev ka müraste andmetega.

Tuletame meelde, et hüperparameetrite tuunimiseks treenime erinevate hüperparameetrite kombinatsioonidega mudeleid ning testime neid valideerimisandmestikul. Valituks osutub mudel hüperparameetrite kombinatsiooniga, mis annab valideerimisandmestikul parimaid tulemusi.

Teoorias võib üle kombinatsioonide itereerida käsitsi, kuid *scikit-learn*'is on selleks ka [lahendused olemas](http://scikit-learn.org/stable/modules/grid_search.html). Hüperparameetrite valikuks kasutatakse tihti kas *grid search*'i, mis leiab etteantud hüperparameetrite kõikvõimalikud kombinatsioonid, või juhuslikke protsesse, kui hüperparameetrite jaotused on ette antud, valides jaotustest juhuslikult mingi hulga kombinatsioone.

Treenime "optimaalsete" hüperparameetritega juhusliku metsa klassifitseerija. Kasutame *grid search*'i. *Scikit-learn*'i *grid search*'ile antakse hüperparameetrite testitavad väärtused sõnaraamatuna. Kuivõrd juhusliku metsa klassifitseerija hüperparameetrid on näiteks *n_estimators* (puude arv metsas), *max_features* (palju tunnuseid ühes puus on maksimaalselt) ja *max_depth* (kui sügavad puud tehakse).

In [8]:
hyper_parameters = {'n_estimators':(1,10,20), 'max_features':(1,5,10), 'max_depth':(1,3,7)}

Kirjeldatud parameetreid võib julgesti muuta ning varieerida võib ka teisi [dokumentatsiooni](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) järgi.

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier()
clf = GridSearchCV(rfc,hyper_parameters)

training_features = None
training_labels = None

clf.fit(training_features, training_labels)

Paneme tähele, et *GridSearchCV* tuletab valideerimisandmestiku (*hold-out set*'i) kõhus treeningandmestikust osa eraldades, mistõttu valideerimisandmestiku loomine pole antud klassi kasutades oluline (ehkki praktikas on hea teada).

Järgmiseks kuvame parima hüperparameetrite kombinatsiooni ning selle tulemuse automaatselt loodud valideerimisandmestikul. Selleks saab kasutada näiteks

In [ ]:
from sklearn.metrics import classification_report

clf.best_params_
dev_features = None
dev_labels = None
y_true, y_pred = dev_labels, clf.predict(dev_features)
print(classification_report(y_true, y_pred, target_names=['klass1', 'klass2', 'klass3'])

Võrdleme vaikehüperparameetritega treenitud juhuslikku metsa parima leitud hüperparameetrite kombinatsiooniga treenituga. Võime kasutada näiteks F1 skoori.

#### Tugivektormasin (Support Vector Machine) (1p)
Treenime sarnaselt juhuslikule metsale "parima" hüperparameetrite kombinatsiooniga klassifitseerimismudeli, tuvastame parima kombinatsiooni ja kvaliteedi valideerimisandmestikul ning võrdleme loodut vaikehüperparameetritega versiooniga. Kindlasti võiks proovida varieerida *kernel*'eid (*rbf*, *linear*, *poly*, *sigmoid*), kuid teiste hüperparameetritega saab tutvuda [dokumentatsioonis](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

#### Otsustuspuu (decision tree) (0,5p)
Treenida vaikeparameetritega [otsustuspuu](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

#### Võrrelda erinevatel meetoditel saadud klassifitseerijate tulemusi testandmestikul (0,5p)

Võrrelda parima juhusliku metsa, tugivektormasina ja otsustuspuu F1 skoore ning väljastada võrdlus

### Mitmene klassifitseerimine (1p)
Jätke reiting skaalale 1-5. Ärge klasse tasakaalustage. Jagage andmestik treening- ja testandmestikuks ning treenige klassifitseerimismudel meelepärase meetodiga. Kirjeldage testandmestikul saadud tulemusi eelmiste tulemuste taustal (kvaliteedi skoori võrdlus, klasside täpsused, püüdke põhjendada).

### Kas jäime rahule meelestatuse tulemusega?
Ilmselt mitte. Antud juhul oli väike, mürane ja kallutatud andmestik, kuid ka reaalsetes olukordades ei piisa nii naiivsetest tunnustest. Meelestatuses kasutatakse sageli ettevalmistatud leksikone või ontoloogiaid (nt WordNet). Teksti meelestatuse juures mängivad rolli nii "põhilised" sõnad kui ka nende modifitseerijad. Kui sõna "ilus" on lauses tugeva signaaliga, siis lause lõpliku meelestatuse võivad muuhulgas määrata näiteks täiendavad sõnad "väga ilus", "ei ole väga ilus", "ei ole ilus", "#!%$& ilus".

### Boonusülesanne 3. Tutvuge meelestatuse analüüsi trikkidega näiteks [siin](http://sentiment.christopherpotts.net/) ning rakendage neid eelmisele andmestikule, treenides mingi meelepärase mudeli. Kirjeldage ideed ning võrrelge, kas lahendus paranes. (0,5p)

### Valmis olevad lahendused
Meelestatuse analüüsi ei ole alati ilmtingimata ise tarvis implementeerida. Meelestatuse tuvastamist pakuvad ka erinevad teegid, millest mõjuvõimsaim on ilmselt [NLTK](http://www.nltk.org/howto/sentiment.html). Samas tuleb olla ettevaatlik, kuna meelestatuses mängib rolli semantika ning inglise keelel hästi töötav lahendus, mis kasutab veel näiteks ingliskeelseid sõnastikke, ei tööta kindlasti hästi eesti keelel.

## Mustrite tuvastamine juhendatud õppes

Üha enam nõutakse, et masinõppemeetodeid saaks tõlgendada. See võimaldab nii paremat kvaliteedi hindamist, mudeli parandamise võimaluste tuvastamist kui ka abistab domeenieksperte, kirjeldades protsesse, mida võib teistel puhkudel olla raske märgata. Võime mõelda näiteks äriprotsesside tulemuste tuvastamisele või protsesside optimeerimisele. Analüütikul (või ka arstil) on tarvis veenduda, et pakutud lahendus on realistlik ning arutluskäik võtab arvesse olulisi tunnuseid või sümptomeid.

Oma olemuselt on üheks kõige paremini lahenduskäiku kirjeldavaks viisiks otsustuspuud.

## Ül 2. Probleemi kirjeldamine otsustuspuu baasil (1p) 

**Eesmärk.** Leida, millised tekstilised tunnused mõjutavad enim reitingut.

**Lugeda ja töödelda andmed.**
* JÄTTA REITING SKAALALE 1-5,
* jagada andmestik treening- ja testandmestikuks (nt 80%-20%).

**Treenida otsustuspuu ja juhusliku metsa klassifitseerija.**
* treenida treeningandmestikul vaikeparameetritega
     * otsustuspuu klassifitseerija
     * juhusliku metsa klassifitseerija

** Leida kõige olulisemad tunnused mõlemal klassifitseerijal:**
* sorteerida tunnused olulisuse alusel
     * nii otsustuspuu kui juhuslik mets talletavad iga tunnuse olulisuse astaku
         * model.feature_importances_
     * sorteerimiseks on otstarbekas kasutada Numpy *array argsort*’i, mis tagastab sorteeritud elementide asemel vastavad indeksid
         * kõige ebaolulisema tunnuse indeks on esimene, kõige olulisema tunnuse indeks viimane
* valida mingi arv kõige olulisemaid tunnuste indekseid ning seada neile ülesande 1A-ga konsulteerides mõistlikud nimed, näiteks
      * “‘hea’ sagedus”
      * “kommentaari pikkus”
* võrrelda klassifitseerijate pakutud olulisemaid tunnuseid
* kumma tunnuseid usaldate enam ja miks?

## Ül. 3 Vead mudelis (1,5p)
**Motivatsioon.** Masinõppemeetodid annavad kehvi tulemusi erinevatel põhjustel. Käesolevas ülesandes tutvume kolme sagedase juhuga.

**Eesmärk.** Simuleerida erinevaid stsenaariumeid, mil masinõppemeetodid annavad kehvi tulemusi.

**Luua optimaalne *baseline* mudel, mida hakkame kehvade mudelitega võrdlema.**
* lugeda sisse kettale salvestatud Numpy struktuurid,
* jagada andmestik treening- ja testandmestikuks (nt 80-20)
* treenida treeningandmestikul vaikeparameetritega otsustuspuu
* leida testandmestikul otsustuspuu *accuracy*

**Liiga lihtne mudel. Alaõppimine. (0,5p)** 
* treenida treeningandmestikul otsustuspuu
     * limiteerida maksimaalsete tunnuste arvu (*max_features*)
         * sõltuvalt oluliste tunnuste arvust (võib võtta nt 2-4)
* leida testandmestikul otsustuspuu ​ accuracy
* võrrelda *baseline*’iga

**Puuduvad olulised tunnused. (0,5p)**
* tuvastada *baseline*’i abil *n* kõige olulisemat tunnust
* treenida treeningandmestikul otsustuspuu ilma eelnevalt leitud *n* parima tunnuseta
    * tunnused võib välja jätta *masked_array*’d kasutades
         * np.ma
* leida testandmestikul otsustuspuu *accuracy*
* võrrelda *baseline*’iga

**Mudel liiga väheste andmetega. (0,5p)**
* jagada andmestik treening- ja testandmestikuks (10%-90%)
* treenida treeningandmestikul vaikeparameetritega otsustuspuu
* leida testandmestikul otsustuspuu *accuracy*
* võrrelda *baseline*’iga

## OCR (Optical Character Recognition)

Keele ja juhendatud õppega on tugevalt seotud ka kirjakeele tuvastamine ja digitaliseerimine. Näiteks saame analüüsida vanavanemate kirjavahetust või antiikseid ürikuid. Pildituvastussüsteemi implementeerimine vajab teadmisi ja oskusi, mis ei mahu käesoleva aine raamesse, kuid Püütonis on olemas teegid, mis võimaldavad ilma suurema *know-how*'ta lihtsamaid probleeme lahendada. Head tagasisidet on saanud näiteks [PyTesseract](https://pypi.python.org/pypi/pytesseract), mis on Püütoni *wrapper* Google'i *Tesseract-OCR* API-le.

### Ülesanne 4. OCR
Kasutage *PyTesseract*'i teeki, et proovida tuvastada/digitaliseerida teksti mingisuguselt pildilt. Lihtsuse mõttes tasuks valida pilt, millel on valge taust ning tekst on otse. Lahendusena kood, pilt *Notebook*'is ning pildilt tuvastatud tekst. (1p)

In [24]:
counts = {'ja':5, 'tere':3, 'maagia':15, 'koer':1}

counts_list = [(word, counts[word]) for word in counts]
print(counts_list)
print(sorted(counts_list, key = lambda x: x[1]))

print([(word, counts[word]) for word in sorted(counts, key = lambda x: -counts[x])])


def f(x):
    return x[1]

[('koer', 1), ('ja', 5), ('tere', 3), ('maagia', 15)]
[('koer', 1), ('tere', 3), ('ja', 5), ('maagia', 15)]
[('maagia', 15), ('ja', 5), ('tere', 3), ('koer', 1)]
